<a href="https://colab.research.google.com/github/richieri/notebooks/blob/main/Chapter_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The idea

Extract possible chapters from subtitles of a video. For the purporse of this colab, we are going to use the "Machine Learning for Everybody" from freeCodeCamp.org:

https://www.youtube.com/watch?v=i_LwzRVP7bg

# How to use it

1.   Upload your VTT caption file to the root of your colab environment with the name "captions.vtt"
2.   Run the code :)

# How to get a captions.vtt file

If you don't have the captions file yet, you can generate them at https://savesubs.com/ . Got to their sites, paste the youtube video URL in the main box and ask for download.

After generating it, Savesubs will give a few different formats of subtitles, pick VTT.

File format must be something like this:

```
WEBVTT

00:00:00.000 --> 00:00:06.000
Kylie Ying has worked at many interesting places such as MIT, CERN, and Free Code Camp.

00:00:06.000 --> 00:00:10.880
She's a physicist, engineer, and basically a genius. And now she's going to teach you

00:00:10.880 --> 00:00:14.720
about machine learning in a way that is accessible to absolute beginners.

00:00:15.280 --> 00:00:21.600
What's up you guys? So welcome to Machine Learning for Everyone. If you are someone who


```

# Author

Ronaldo Richieri

https://github.com/richieri

# Code explained

Install required dependencies

In [ ]:
!pip install langchain openai tiktoken

Import libraries and set API Key

In [ ]:
from langchain import OpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import PromptTemplate

import os

openai_api_key = "PLACE YOUR OPEN AI API KEY HERE"

Ingest the captions file to a "captions" variable

In [ ]:
video_captions = './captions.vtt'

with open(video_captions, 'r') as file:
    captions = file.read()

Split the captions file in multiple chunks that will be stuffed into the prompt to be sent to OpenAI. This is required because our calls to OpenAI have a limit of words (tokens to be more accurated) that can be sent.
Which model has its own limit. In this case, we are working with a of 4096 tokens.

We split in multiple documents.

In [ ]:
llm = OpenAI(temperature=0, openai_api_key=openai_api_key)

llm.get_num_tokens(captions)

text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n"], chunk_size=5000, chunk_overlap=400)

docs = text_splitter.create_documents([captions])

num_docs = len(docs)
num_tokens_first_doc = llm.get_num_tokens(docs[0].page_content)
print (f"Now we have {num_docs} documents and the first one has {num_tokens_first_doc} tokens")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


Now we have 50 documents and the first one has 1554 tokens


Prepare the prompt

In [ ]:
template = """
Summarize the following SBV subtitles from a video in one single topic and add the time it started and ended, returning the output in only this JSON format {{"start":"00:00:00","end":"00:00:00","topic":"..."}}:
{doc}
"""

prompt = PromptTemplate(
    input_variables=["doc"],
    template=template
)


Send each chuk (stuffed into the prompt) to OpenAI and get the results

In [ ]:
for doc in docs:
  # print("This doc",doc.page_content)
  summary_prompt = prompt.format(doc=doc.page_content)
  num_tokens = llm.get_num_tokens(summary_prompt)
#  print (f"This prompt + doc has {num_tokens} tokens")
  summary = llm(summary_prompt)
  print (summary.strip())

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{"start":"00:00:00","end":"00:04:23.600","topic":"Introduction to Machine Learning and Using the UCI Machine Learning Repository"}
{"start":"00:04:00.960","end":"00:09:01.360","topic":"Pandas and Data Frames"}
{"start":"00:08:43.360","end":"00:13:07.440","topic":"Machine Learning and its Types"}
{"start":"00:12:48.400","end":"00:17:19.680","topic":"Types of Data and One Hot Encoding"}
{"start":"00:17:02.080","end":"00:21:23.760","topic":"Supervised Learning and Model Evaluation"}
{"start":"00:21:04.240","end":"00:25:45.920","topic":"Supervised Learning and Model Training"}
{"start":"00:25:25.680","end":"00:30:11.360","topic":"Loss Functions and Performance Measures"}
{"start":"00:29:47.840","end":"00:34:34.640","topic":"Measuring Model Performance and Data Visualization"}
{"start":"00:34:11.200","end":"00:39:20.400","topic":"Data Scaling and Stacking"}
{"start":"00:38:58.400","end":"00:44:23.120","topic":"Oversampling and balancing training data"}
{"start":"00:43:59.680","end":"00:49:2